### Exercício 01 - EcoRecipiente S.A.



In [1]:
from pulp import *

demanda = {
    (35, 1, 1): 138, (35, 1, 2): 142, (35, 1, 3): 139, (35, 1, 4): 140,
    (35, 2, 1): 82,  (35, 2, 2): 84,  (35, 2, 3): 83,  (35, 2, 4): 84,
    (35, 3, 1): 32,  (35, 3, 2): 33,  (35, 3, 3): 34,  (35, 3, 4): 36,
    (35, 4, 1): 61,  (35, 4, 2): 66,  (35, 4, 3): 67,  (35, 4, 4): 75,
    (42, 1, 1): 284, (42, 1, 2): 278, (42, 1, 3): 305, (42, 1, 4): 322,
    (42, 2, 1): 226, (42, 2, 2): 255, (42, 2, 3): 272, (42, 2, 4): 289,
    (42, 3, 1): 141, (42, 3, 2): 160, (42, 3, 3): 175, (42, 3, 4): 188,
    (42, 4, 1): 134, (42, 4, 2): 116, (42, 4, 3): 126, (42, 4, 4): 130,
    (42, 5, 1): 1168, (42, 5, 2): 1138, (42, 5, 3): 1204, (42, 5, 4): 1206
}

maquinas_dados = {
    ('D5', 1): {'tipos': [35, 42], 'custo': {35: 760, 42: 454}, 'dias_maq': {35: 0.097, 42: 0.037}},
    ('D6', 1): {'tipos': [35, 42], 'custo': {35: 879, 42: 476}, 'dias_maq': {35: 0.103, 42: 0.061}},
    ('C4', 1): {'tipos': [35, 42], 'custo': {35: 733, 42: 529}, 'dias_maq': {35: 0.080, 42: 0.068}},
    ('T', 2): {'tipos': [42], 'custo': {42: 520}, 'dias_maq': {42: 0.043}},
    ('U2', 2): {'tipos': [35, 42], 'custo': {35: 790, 42: 668}, 'dias_maq': {35: 0.109, 42: 0.056}},
    ('K4', 3): {'tipos': [35, 42], 'custo': {35: 758, 42: 509}, 'dias_maq': {35: 0.119, 42: 0.061}},
    ('J6', 3): {'tipos': [35, 42], 'custo': {35: 799, 42: 521}, 'dias_maq': {35: 0.119, 42: 0.056}},
    ('70', 3): {'tipos': [35, 42], 'custo': {35: 888, 42: 625}, 'dias_maq': {35: 0.140, 42: 0.093}},
    ('1', 4): {'tipos': [35, 42], 'custo': {35: 933, 42: 538}, 'dias_maq': {35: 0.113, 42: 0.061}},
    ('V1', 5): {'tipos': [42], 'custo': {42: 503}, 'dias_maq': {42: 0.061}}
}

disponibilidade = {
    ('C4', 1): 88, ('C4', 2): 89, ('C4', 3): 89, ('C4', 4): 88,
    ('D5', 1): 88, ('D5', 2): 89, ('D5', 3): 89, ('D5', 4): 88,
    ('D6', 1): 72, ('D6', 2): 63, ('D6', 3): 58, ('D6', 4): 65,
    ('U2', 1): 81, ('U2', 2): 88, ('U2', 3): 87, ('U2', 4): 55,
    ('T', 1): 88, ('T', 2): 75, ('T', 3): 89, ('T', 4): 88,
    ('K4', 1): 88, ('K4', 2): 89, ('K4', 3): 89, ('K4', 4): 88,
    ('J6', 1): 37, ('J6', 2): 89, ('J6', 3): 39, ('J6', 4): 86,
    ('70', 1): 54, ('70', 2): 84, ('70', 3): 85, ('70', 4): 73,
    ('1', 1): 42, ('1', 2): 71, ('1', 3): 70, ('1', 4): 68,
    ('V1', 1): 88, ('V1', 2): 89, ('V1', 3): 89, ('V1', 4): 88
}

custo_transporte = {
    (1, 2): 226, (1, 3): 274, (1, 4): 93, (1, 5): 357,
    (2, 1): 226, (2, 3): 371, (2, 4): 310, (2, 5): 443,
    (3, 1): 274, (3, 2): 371, (3, 4): 227, (3, 5): 168,
    (4, 1): 93, (4, 2): 310, (4, 3): 227, (4, 5): 715,
    (5, 1): 357, (5, 2): 443, (5, 3): 168, (5, 4): 715
}

capacidade_arm = {
    (1, 1): 376, (1, 2): 325, (1, 3): 348, (1, 4): 410,
    (2, 1): 55, (2, 2): 47, (2, 3): 62, (2, 4): 58,
    (3, 1): 875, (3, 2): 642, (3, 3): 573, (3, 4): 813,
    (4, 1): 10, (4, 2): 15, (4, 3): 30, (4, 4): 24,
    (5, 1): 103, (5, 2): 103, (5, 3): 30, (5, 4): 410
}

custo_manuseio = {
    (35, 1): 85, (35, 2): 98, (35, 3): 75, (35, 4): 90,
    (42, 1): 70, (42, 2): 98, (42, 3): 75, (42, 4): 80, (42, 5): 67
}

modelo = LpProblem("EcoRecipiente_Otimizacao", LpMinimize)

print("DEFININDO VARIÁVEIS DE DECISÃO...")

x = {}
for (maq, planta) in maquinas_dados.keys():
    for tipo in maquinas_dados[(maq, planta)]['tipos']:
        for t in range(1, 5):
            x[(maq, tipo, t)] = LpVariable(f"prod_{maq}_tipo{tipo}_t{t}", lowBound=0)

y = {}
for tipo in [35, 42]:
    for t in range(1, 5):
        for (p1, p2) in custo_transporte.keys():
            if tipo == 35 and (p1 == 5 or p2 == 5):
                continue
            y[(tipo, p1, p2, t)] = LpVariable(f"trans_tipo{tipo}_p{p1}_p{p2}_t{t}", lowBound=0)

s = {}
for tipo in [35, 42]:
    for p in range(1, 6):
        if tipo == 35 and p == 5:
            continue
        for t in range(1, 5):
            s[(tipo, p, t)] = LpVariable(f"estoque_tipo{tipo}_p{p}_t{t}", lowBound=0)

print("DEFININDO FUNÇÃO OBJETIVO...")

custo_total = 0

for (maq, planta) in maquinas_dados.keys():
    for tipo in maquinas_dados[(maq, planta)]['tipos']:
        for t in range(1, 5):
            custo_total += maquinas_dados[(maq, planta)]['custo'][tipo] * x[(maq, tipo, t)]

for tipo in [35, 42]:
    for t in range(1, 5):
        for (p1, p2) in custo_transporte.keys():
            if tipo == 35 and (p1 == 5 or p2 == 5):
                continue
            if (tipo, p1, p2, t) in y:
                custo_total += custo_transporte[(p1, p2)] * y[(tipo, p1, p2, t)]

for tipo in [35, 42]:
    for p in range(1, 6):
        if tipo == 35 and p == 5:
            continue
        for t in range(1, 5):
            if (tipo, p, t) in s:
                custo_total += custo_manuseio[(tipo, p)] * s[(tipo, p, t)] * 2  # entrada + saída

modelo += custo_total

print("ADICIONANDO RESTRIÇÕES...")

for (maq, planta) in maquinas_dados.keys():
    for t in range(1, 5):
        if (maq, t) in disponibilidade:
            dias_usados = 0
            for tipo in maquinas_dados[(maq, planta)]['tipos']:
                dias_usados += maquinas_dados[(maq, planta)]['dias_maq'][tipo] * x[(maq, tipo, t)]
            modelo += dias_usados <= disponibilidade[(maq, t)], f"capacidade_{maq}_t{t}"

for tipo in [35, 42]:
    for p in range(1, 6):
        if tipo == 35 and p == 5:
            continue
        for t in range(1, 5):
            producao = 0
            for (maq, planta) in maquinas_dados.keys():
                if planta == p and tipo in maquinas_dados[(maq, planta)]['tipos']:
                    producao += x[(maq, tipo, t)]
            
            estoque_anterior = 0
            if t > 1 and (tipo, p, t-1) in s:
                estoque_anterior = s[(tipo, p, t-1)]
            
            trans_recebido = 0
            for p_orig in range(1, 6):
                if (p_orig, p) in custo_transporte and (tipo, p_orig, p, t) in y:
                    trans_recebido += y[(tipo, p_orig, p, t)]
            
            demanda_atual = demanda.get((tipo, p, t), 0)
            
            estoque_atual = 0
            if (tipo, p, t) in s:
                estoque_atual = s[(tipo, p, t)]
            
            trans_enviado = 0
            for p_dest in range(1, 6):
                if (p, p_dest) in custo_transporte and (tipo, p, p_dest, t) in y:
                    trans_enviado += y[(tipo, p, p_dest, t)]
            
            modelo += (producao + estoque_anterior + trans_recebido == 
                      demanda_atual + estoque_atual + trans_enviado), f"balanco_tipo{tipo}_p{p}_t{t}"

for p in range(1, 6):
    for t in range(1, 5):
        estoque_total = 0
        for tipo in [35, 42]:
            if tipo == 35 and p == 5:
                continue
            if (tipo, p, t) in s:
                estoque_total += s[(tipo, p, t)]
        if (p, t) in capacidade_arm:
            modelo += estoque_total <= capacidade_arm[(p, t)], f"capacidade_arm_p{p}_t{t}"

print("RESOLVENDO O MODELO...")

modelo.solve(PULP_CBC_CMD(msg=0))

print(f"\nSTATUS DA SOLUÇÃO: {LpStatus[modelo.status]}")

if modelo.status == 1:  
    print(f"CUSTO TOTAL MÍNIMO: ${value(modelo.objective):,.2f}")
    
    print("\n=== PLANO DE PRODUÇÃO ===")
    for (maq, planta) in maquinas_dados.keys():
        print(f"\nPlanta {planta} - Máquina {maq}:")
        for tipo in maquinas_dados[(maq, planta)]['tipos']:
            for t in range(1, 5):
                if x[(maq, tipo, t)].varValue > 0.1:
                    print(f"  Tipo {tipo}, Trimestre {t}: {x[(maq, tipo, t)].varValue:.1f} unidades")
    
    print("\n=== TRANSPORTE ===")
    for tipo in [35, 42]:
        print(f"\nTipo {tipo}:")
        for t in range(1, 5):
            print(f"  Trimestre {t}:")
            for (p1, p2) in custo_transporte.keys():
                if tipo == 35 and (p1 == 5 or p2 == 5):
                    continue
                if (tipo, p1, p2, t) in y and y[(tipo, p1, p2, t)].varValue > 0.1:
                    print(f"    Planta {p1} → Planta {p2}: {y[(tipo, p1, p2, t)].varValue:.1f} unidades")
    
    print("\n=== ESTOQUE ===")
    for tipo in [35, 42]:
        print(f"\nTipo {tipo}:")
        for p in range(1, 6):
            if tipo == 35 and p == 5:
                continue
            print(f"  Planta {p}:")
            for t in range(1, 5):
                if (tipo, p, t) in s and s[(tipo, p, t)].varValue > 0.1:
                    print(f"    Trimestre {t}: {s[(tipo, p, t)].varValue:.1f} unidades")

else:
    print("PROBLEMA NÃO RESOLVIDO OU INVIÁVEL")


print("\n=== RESUMO DAS RESTRIÇÕES ===")
print(f"Total de variáveis: {len(modelo.variables())}")
print(f"Total de restrições: {len(modelo.constraints)}")


DEFININDO VARIÁVEIS DE DECISÃO...
DEFININDO FUNÇÃO OBJETIVO...
ADICIONANDO RESTRIÇÕES...
RESOLVENDO O MODELO...

STATUS DA SOLUÇÃO: Optimal
CUSTO TOTAL MÍNIMO: $5,061,339.00

=== PLANO DE PRODUÇÃO ===

Planta 1 - Máquina D5:
  Tipo 42, Trimestre 1: 284.0 unidades
  Tipo 42, Trimestre 2: 278.0 unidades
  Tipo 42, Trimestre 3: 305.0 unidades
  Tipo 42, Trimestre 4: 322.0 unidades

Planta 1 - Máquina D6:

Planta 1 - Máquina C4:
  Tipo 35, Trimestre 1: 199.0 unidades
  Tipo 35, Trimestre 2: 208.0 unidades
  Tipo 35, Trimestre 3: 206.0 unidades
  Tipo 35, Trimestre 4: 215.0 unidades

Planta 2 - Máquina T:
  Tipo 42, Trimestre 1: 226.0 unidades
  Tipo 42, Trimestre 2: 255.0 unidades
  Tipo 42, Trimestre 3: 272.0 unidades
  Tipo 42, Trimestre 4: 289.0 unidades

Planta 2 - Máquina U2:
  Tipo 35, Trimestre 1: 82.0 unidades
  Tipo 35, Trimestre 2: 84.0 unidades
  Tipo 35, Trimestre 3: 83.0 unidades
  Tipo 35, Trimestre 4: 84.0 unidades

Planta 3 - Máquina K4:
  Tipo 35, Trimestre 1: 32.0 unidade